In [1]:
# Import packages and libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math


import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from statsmodels.tsa.stattools import adfuller  # test for stationarity
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf  # determine p and q arguments of ARIMA(p,q,d)

from statsmodels.tsa.seasonal import seasonal_decompose

from statsmodels.tsa.arima.model import ARIMA  # model

from pmdarima.arima import auto_arima
from pmdarima.arima.utils import ndiffs

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error



In [2]:
# Reading in the data
data = pd.read_csv('logarithmic_7_adj_close.csv')
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'logarithmic_7_adj_close.csv'

In [ ]:
# Selecting sub-set of features
df= data[['timestamp', 'msft_adj_close']]
df

In [ ]:
# Convert the 'Date' column to datetime format
df['timestamp']= pd.to_datetime(df['timestamp'])
df.info()

In [ ]:
# Set Index 
df.set_index('timestamp',inplace=True)

In [ ]:
df

In [ ]:
# Resample Data to Monthly instead of Daily by Aggregating Using Mean
monthly_avg = df['msft_adj_close'].resample('M').mean()
monthly_avg

In [ ]:
monthly_avg = monthly_avg.to_frame()
monthly_avg

In [ ]:
plt.plot(monthly_avg)
plt.show()

### 2. Check for stationarity - Augmented Dickey Fuller (ADF) test 

The null hypothesis of the ADF test is 
that the time series is non-stationary.

if p-value < 0.05 then series is stationary
if p-value > 0.05 then series in non-stationary

#### in our case, p-value = 0.850147, hence series is non-stationary



In [ ]:
# Checking whether series is stationary using Augmented
# Dickey Fuller (ADF) test

result = adfuller(monthly_avg.dropna())
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])


In [ ]:
# Differenced data
prices_diff = monthly_avg.diff().dropna()

# ADF test
adf_res_diff_data = adfuller(prices_diff)
print(f'ADF Statistic: {adf_res_diff_data[0]}')
print(f'p-value:  {adf_res_diff_data[1]}')



In [ ]:
# Plot original and differenced data
plt.plot(monthly_avg, 'blue', label = 'adjusted close')
plt.plot(monthly_avg.diff().dropna(), 'red', label = 'adjusted close 1st differencing') 

plt.legend()
plt.show()


#### * We can also make use of .ndiffs() method from pmdadima

In [ ]:
# Using .ndiffs() method from pmdadima to determine the minimum number of differencing required to 
# make data stationary
d = ndiffs(monthly_avg, test = 'adf')
print(f'The number of differencing required for this data to be stationary is {d}.')

### 3. Explore (separate) trend and seasonality

In [ ]:
plt.rcParams["figure.figsize"] = (16,9)

result = seasonal_decompose(monthly_avg['msft_adj_close'], model = 'additive', period = 20)  #model = 'multiplicative'
result.plot()

plt.show()

### 4. Determine order for AR component 'p' using partial autocorrelation plot


In [ ]:
# Set a variable with differenced data
diff_data = monthly_avg.msft_adj_close.diff().dropna()

# PACF  
plot_pacf(diff_data)
plt.ylim(-0.25, 1.1)
plt.savefig('PACF.pdf')

# ACF
plot_acf(diff_data)
plt.ylim(-0.25, 1.1)
plt.savefig('ACF.pdf')
plt.show()

### 5. Splitting tha data into train and test 


In [ ]:
rows = int(len(monthly_avg['msft_adj_close']) * 0.80)
rows

In [ ]:
train = monthly_avg['msft_adj_close'][:rows]
test = monthly_avg['msft_adj_close'][rows:]

In [ ]:
train

In [ ]:
test.shape

In [ ]:
# Use the following to determine optimal order for ARIMA components

In [ ]:
p_values = [0, 4,10, 11]
d_values = range(0, 2)
q_values = [0, 4,10, 11]

In [ ]:
for p in p_values:
    for d in d_values:
        for q in q_values:
            order = (p,d,q)
            # warnings.filterwarnings("ignore")
            model = ARIMA(train, order=order).fit()
            predictions = model.predict(start=len(train), end=len(train) + len(test) - 1)
            error = mean_squared_error(test, predictions)
            print('ARIMA%s MSE=%.3f' % (order,error))
        
# best so far - ARIMA(0, 1, 0) MSE=654.110

### 6. Implementing ARIMA(p,d,q) model

     6.1. Training the model

In [ ]:
# ARIMA(p,d,q) model
model = ARIMA(train, order=(11,0,4))
model_fit = model.fit()
print(model_fit.summary())

     6.1. Testing the model

In [ ]:
y_pred = model_fit.predict(start = rows + 1, end = len(monthly_avg.msft_adj_close)) #, dynamic=True)
print(f'ARIMA Model Test Data MSE: {np.mean((y_pred.values - test.values)**2):.3f}')
y_pred

     6.1. Evaluating the model

In [ ]:
################################################################################
# Checking the accuracy of the model.
# r^2, Root Mean Square Error, Mean Absolute Error
from math import sqrt
score = r2_score(test, y_pred)
mse = mean_squared_error(test, y_pred)
# mse = np.mean((y_pred.values - test.values)**2)
rmse = sqrt(mean_squared_error(test, y_pred))
mae = mean_absolute_error(test, y_pred)


print("\n R^2 score is: {:.6f}".format(score)) # negative when the model does not
# print("\n The MSE is: {:.6f}".format(mse1))
print("\n The MSE is: {:.6f}".format(mse))
print("\n The RMSE is: {:.6f}".format(rmse))
print("\n The MAE: {:.6f} ".format(mae))

In [ ]:
# Visualising log data
plt.plot(train, 'blue', label = 'train')
plt.plot(test, 'green', label = 'test' )
plt.plot(y_pred, 'purple', label = 'predictions')
# plt.xlim(3700, 4010)
plt.legend(loc='upper left')
plt.savefig('Adjusted_Close_Monthly_Average_Prices.pdf')

plt.show()

In [ ]:
# # Visualising log data
# plt.plot(train, 'blue', label = 'train')
# plt.plot(test, 'green', label = 'test' )
# plt.plot(y_pred, 'purple', label = 'predictions')
# # plt.xlim(3700, 4010)
# plt.legend()

# plt.show()

### 6. Implementing SARIMA(p,d,q)(P,D,Q,M) model

In [ ]:
# from statsmodels.tsa.statespace.sarimax import SARIMAX
# from itertools import product

In [ ]:
# best_model = SARIMAX(train, order=(11, 0, 4), seasonal_order=(2, 0, 1, 12)).fit(dis=-1)
# # best_model.summary()

In [ ]:
# y_pred = model_fit.predict(start = rows+1, end = len(monthly_avg.msft_adj_close)) #, dynamic=True)
# print(f'SARIMA Model Test Data MSE: {np.mean((y_pred.values - test.values)**2):.3f}')
# # y_pred

In [ ]:
# ################################################################################
# # Checking the accuracy of the model.
# # r^2, Root Mean Square Error, Mean Absolute Error
# from math import sqrt
# score = r2_score(test, y_pred)
# mse = mean_squared_error(test, y_pred)
# # mse = np.mean((y_pred.values - test.values)**2)
# rmse = sqrt(mean_squared_error(test, y_pred))
# mae = mean_absolute_error(test, y_pred)


# print("\n R^2 score is: {:.6f}".format(score)) # negative when the model does not
# # print("\n The MSE is: {:.6f}".format(mse1))
# print("\n The MSE is: {:.6f}".format(mse))
# print("\n The RMSE is: {:.6f}".format(rmse))
# print("\n The MAE: {:.6f} ".format(mae))

order=(0, 1, 0), seasonal_order=(1, 0, 1, 12) , The MSE is: 304.89977
order=(0, 1, 0), seasonal_order=(2, 0, 1, 12) , The MSE is: 304.899778

In [ ]:
# # Visualising log data
# plt.plot(train, 'blue', label = 'train')
# plt.plot(test, 'green', label = 'test' )
# plt.plot(y_pred, 'purple', label = 'predictions')
# # plt.xlim(3700, 4010)
# plt.legend()

# plt.show()

In [ ]:

# import itertools
# # warnings.filterwarnings("ignore")

In [ ]:
# p = d = q = range(0, 3)
# pdq = list(itertools.product(p, d, q))
# seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

# print('Examples of grid search Model parameter combinations for Seasonal-ARIMA')
# print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
     

In [ ]:
# list_param = []
# list_param_seasonal=[]
# list_results_aic=[]

# for param in pdq:
#     for param_seasonal in seasonal_pdq:
#         try:
#             model = SARIMAX(train, order=param,
#                                             seasonal_order=param_seasonal,
#                                             enforce_stationarity=False,
#                                             enforce_invertibility=False)

#             results = model.fit()

#             print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
            
#             list_param.append(param)
#             list_param_seasonal.append(param_seasonal)
#             list_results_aic.append(results.aic)
#         except:
#             continue

In [ ]:
# list_param = []
# list_param_seasonal=[]
# list_results_aic=[]

# for param in pdq:
#     for param_seasonal in seasonal_pdq:
#         model = SARIMAX(train,
#                                         order=param,
#                                         seasonal_order=param_seasonal,
#                                         enforce_stationarity=False,
#                                         enforce_invertibility=False)

#         results = model.fit()

#         print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))

#         list_param.append(param)
#         list_param_seasonal.append(param_seasonal)
#         list_results_aic.append(results.aic)
